In [ ]:
%pip install tf-keras pandas deepface opencv-python

In [ ]:
import threading
import cv2
from deepface import DeepFace
import json
import pandas as pd

In [ ]:
verstappen1="./image_database/Max Verstappen/max_verstappen1.jpg"
verstappen2="./image_database/Max Verstappen/max_verstappen2.jpg"
vdraganic="./image_database/Vedran Draganic/vedran_draganic.png"

models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "Dlib", 
  "SFace",
  "GhostFaceNet"
]

#DeepFace demo
result = DeepFace.verify(
  verstappen1,
  vdraganic,
  model_name=models[0]
)
print(json.dumps(result, indent=2))

result = DeepFace.verify(
  verstappen1,
  verstappen2,
  model_name=models[0]
)
print(json.dumps(result, indent=2))

In [7]:
face_match_name = "Unknown"

# Verify face
def check_face(frame):
    global face_match_name
    try:
        results = DeepFace.find(frame, db_path="./image_database")

        if isinstance(results, list) and len(results) > 0:
            results = results[0]

        if isinstance(results, pd.DataFrame) and not results.empty:
            face_match_name = results.iloc[0]['identity'].split("\\")[1]
        else:
            face_match_name = "Unknown"
    except ValueError:
        face_match_name = "Unknown"
    except Exception as e:
        print(f"Error in check_face: {e}")
        face_match_name = "Unknown"


In [ ]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

counter = 0

# Load pre-trained face detector model (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

while True:
    ret, frame = cap.read()

    if ret:
        # Run face matching every 30 frames
        if counter % 30 == 0:
            try:
                threading.Thread(target=check_face, args=(frame.copy(),)).start()
            except Exception as e:
                print(f"Error starting thread: {e}")
                pass
        counter += 1

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        cv2.putText(frame, face_match_name, (20, 450), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 1)

        cv2.imshow("Video", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q") or key == 27: #27 is escape key
        break

    if cv2.getWindowProperty("Video", cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
cv2.destroyAllWindows()